In [1]:
!pip3 install pymupdf pytesseract pillow

/Users/mac/.zshenv:1: command not found: 0x0:0x0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymupdf]m1/2 [pymupdf]


In [ ]:
import fitz  # PyMuPDF
import os
import json

def extract_pdf_content(pdf_path, output_dir="output"):
    # Create output folder if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Open the PDF
    doc = fitz.open(pdf_path)

    full_text = ""
    image_paths = []
    total_images = 0

    for page_num, page in enumerate(doc, start=1):
        # ---- Extract Text ----
        page_text = page.get_text()
        full_text += f"\n--- Page {page_num} ---\n{page_text}"

        # ---- Extract Images ----
        image_list = page.get_images(full=True)
        print(f"Page {page_num} has {len(image_list)} images.")

        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            image_filename = f"page_{page_num}_img_{img_index+1}.{image_ext}"
            image_path = os.path.join(output_dir, image_filename)

            with open(image_path, "wb") as f:
                f.write(image_bytes)

            #image_paths.append(image_path)
            total_images += 1
            print(f"Saved: {image_path}")

    # ---- Save text to a .txt file ----
    text_file = os.path.join(output_dir, "extracted_text.txt")
    with open(text_file, "w", encoding="utf-8") as f:
        f.write(full_text)

    # ---- Save structured metadata to JSON ----
    metadata = {
        "total_pages": len(doc),
        "total_images": total_images,
        "text_file": text_file
    }

    json_file = os.path.join(output_dir, "summary.json")
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=4)

    print("\n✅ Extraction complete.")
    print(f"Text saved to: {text_file}")
    print(f"{total_images} images saved to: {output_dir}")
    print(f"Metadata saved to: {json_file}")
    return metadata

# ==== RUN ====
# Replace this with your actual PDF path
pdf_path = "pdf2.pdf"
extract_pdf_content(pdf_path)


Page 1 has 0 images.

✅ Extraction complete.
Text saved to: output/extracted_text.txt
0 images saved to: output
Metadata saved to: output/summary.json


{'total_pages': 1, 'total_images': 0, 'text_file': 'output/extracted_text.txt'}

In [8]:
pdf_path = "pdf2.pdf"
extract_pdf_content(pdf_path)

Page 1 has 1 images.
Saved: output/page_1_img_1.jpeg
Page 2 has 1 images.
Saved: output/page_2_img_1.jpeg
Page 3 has 2 images.
Saved: output/page_3_img_1.png
Saved: output/page_3_img_2.jpeg

✅ Extraction complete.
Text saved to: output/extracted_text.txt
4 images saved to: output
Metadata saved to: output/summary.json


{'total_pages': 3,
 'total_images': 4,
 'images': ['output/page_1_img_1.jpeg',
  'output/page_2_img_1.jpeg',
  'output/page_3_img_1.png',
  'output/page_3_img_2.jpeg'],
 'text_file': 'output/extracted_text.txt'}